In [1]:
# Artificial Neural Network

# Installing Theano
# pip install --upgrade --no-deps git+git://github.com/Theano/Theano.git

# Installing Tensorflow
# pip install tensorflow

# Installing Keras
# pip install --upgrade keras

# Part 1 - Data Preprocessing

In [2]:
#!pip install -r requirements.txt

In [3]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [4]:
# Importing the dataset
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values

In [5]:
# Encoding categorical data
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
onehotencoder = OneHotEncoder(categorical_features = [1])
X = onehotencoder.fit_transform(X).toarray()
X = X[:, 1:]

/home/naiara/GIT/deeplearningunb/myenv/lib/python3.5/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/home/naiara/GIT/deeplearningunb/myenv/lib/python3.5/site-packages/sklearn/preprocessing/_encoders.py:392: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


In [6]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [7]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [8]:
# Part 2 - Now let's make the ANN!

# Importing the Keras libraries and packages
import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [55]:
# Initialising the ANN
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))

# Adding the second hidden layer
classifier.add(Dense(units = 15, kernel_initializer = 'uniform', activation = 'tanh'))

# Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [56]:
# Fitting the ANN to the Training set
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

Epoch 1/100
8000/8000 [==============================] - 1s 153us/step - loss: 0.4668 - acc: 0.7961
Epoch 2/100
8000/8000 [==============================] - 1s 90us/step - loss: 0.4206 - acc: 0.8129
Epoch 3/100
8000/8000 [==============================] - 1s 81us/step - loss: 0.4130 - acc: 0.8310
Epoch 4/100
8000/8000 [==============================] - 1s 88us/step - loss: 0.4091 - acc: 0.8340
Epoch 5/100
8000/8000 [==============================] - 1s 82us/step - loss: 0.4060 - acc: 0.8342
Epoch 6/100
8000/8000 [==============================] - 1s 83us/step - loss: 0.4044 - acc: 0.8344
Epoch 7/100
8000/8000 [==============================] - 1s 86us/step - loss: 0.4035 - acc: 0.8347
Epoch 8/100
8000/8000 [==============================] - 1s 82us/step - loss: 0.4032 - acc: 0.8327
Epoch 9/100
8000/8000 [==============================] - 1s 83us/step - loss: 0.4022 - acc: 0.8329
Epoch 10/100
8000/8000 [==============================] - 1s 84us/step - loss: 0.4014 - acc: 0.8342
Epoch 11

8000/8000 [==============================] - 1s 85us/step - loss: 0.3491 - acc: 0.8574: 0s - loss: 0.3521 - acc: 0.8
Epoch 83/100
8000/8000 [==============================] - 1s 79us/step - loss: 0.3478 - acc: 0.8565
Epoch 84/100
8000/8000 [==============================] - 1s 79us/step - loss: 0.3472 - acc: 0.8589
Epoch 85/100
8000/8000 [==============================] - 1s 80us/step - loss: 0.3461 - acc: 0.8591
Epoch 86/100
8000/8000 [==============================] - 1s 79us/step - loss: 0.3449 - acc: 0.8581
Epoch 87/100
8000/8000 [==============================] - 1s 79us/step - loss: 0.3436 - acc: 0.8605
Epoch 88/100
8000/8000 [==============================] - 1s 81us/step - loss: 0.3422 - acc: 0.8595
Epoch 89/100
8000/8000 [==============================] - 1s 80us/step - loss: 0.3418 - acc: 0.8612
Epoch 90/100
8000/8000 [==============================] - 1s 80us/step - loss: 0.3404 - acc: 0.8601
Epoch 91/100
8000/8000 [==============================] - 1s 80us/step - loss: 0.34

In [57]:
# Part 3 - Making predictions and evaluating the model

# Predicting the Test set results
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [58]:
cm

array([[1510,   85],
       [ 192,  213]])